### Creating a GUI for recommendation system

In [82]:
import numpy as np
import pandas as pd
import tkinter as tk
import warnings 
warnings.filterwarnings('ignore')


cols = ['user_id','item_id','rating','timestamp']
df = pd.read_csv('u.data',sep='\t',header=None)
df.columns = cols
cols_movie = ['item_id','title']+[str(i) for i in range(22)]
movie_titles = pd.read_csv('u.item',delimiter='|',names= cols_movie,encoding = "ISO-8859-1")
movie_titles = movie_titles[['item_id','title']]
data = pd.merge(df,movie_titles,on='item_id')
rating = pd.DataFrame(data.groupby(by='title')['rating'].mean())
rating['counts'] = data['title'].value_counts()
pivot_df = data.pivot_table(index='user_id', columns = 'title', values = 'rating')


app = tk.Tk()
app.title('Recommendation System')
app.geometry('500x150')

tk.Label(app, text='Movie You have watched:').place(x=25,y=0)

movie_var = tk.Variable(app)
tk.Entry(app, textvariable=movie_var, width = 25).place(x=175,y=0)


def find_recommendation():
    movie = movie_var.get().lower().strip()
    if len(app.winfo_children())>3:
        for widget in app.winfo_children()[3:]:
            widget.destroy()
    if movie:
        try:
            watched = movie_titles['title'][movie_titles['title'].apply(lambda x: movie in x.lower())].values[0]                
        except:
            error = tk.Label(app, text='Movie not found.\n Please try another movie.',  bg='White', fg='Red')
            error.place(x=175,y=50)
            
        else:    
            corr_df = pd.DataFrame(pivot_df.corrwith(pivot_df[watched]), columns = ['Correlation'])
            corr_df.dropna(inplace=True)
            corr_df = corr_df.join(rating['counts'])
            name1 = corr_df[(corr_df['counts']>200) & (corr_df['Correlation']>0.4)].sort_values(by='Correlation',ascending=False).index[1]
            name2 = corr_df[corr_df['Correlation']>0.4].sort_values(by='Correlation',ascending=False).index[1]
            
            rec = tk.Label(app, text='Recommended for you:')
            rec.place(x=25,y=100)         
            recommend_var = tk.Variable(app)
            tk.Label(app, textvariable=recommend_var).place(x=175,y=100)

            atry = tk.Label(app, text='You may also like:')
            atry.place(x=25,y=125)
            alsotry_var = tk.Variable(app)
            tk.Label(app, textvariable=alsotry_var).place(x=175,y=125)
            
            mwatch = tk.Label(app, text= 'Movie watched: ')
            mwatch.place(x=25, y=75)
            name = tk.Variable()
            tk.Label(app, textvariable= name).place(x=175, y=75)
            
            name.set(watched)
            recommend_var.set(name1)
            alsotry_var.set(name2)
    pass
    
tk.Button(app, text='Find Suggestion',
          command= find_recommendation).place(x=175, y= 25)

app.mainloop()